In [1]:
"""
Created on Mon Sep  9 09:33:14 2019

@author: Ming Jin
"""
import os
import json
import time
import datetime
import data_simulator
from itertools import islice
from nupic.data.file_record_stream import FileRecordStream
from htm_anomaly_detection import HTM
from nupic.engine import Network
from nupic.encoders import MultiEncoder, ScalarEncoder, DateEncoder

### Example1. Run HTM network & save model (network status)

In [2]:
'''
parameters that we need to define：
    _TIMEOFDAY: internal buffer, see this: https://nupic.docs.numenta.org/1.0.3/api/algorithms/encoders.html
    _STREAM_BUFFER: The data stream buffer, which decides how many rows of data that should be writed into csv cache file each time
    _USE_SAVED_MODEL: Use saved model or not
'''
_TIMEOFDAY = (21,4)
_STREAM_BUFFER = 10
_USE_SAVED_MODEL = False


'''
Initialize HTM anomaly detection object
'''
htm = HTM(use_saved_model = _USE_SAVED_MODEL)


'''
Run the code below once to set encoders, SP, and TM parameters.
You should define these parameters by yourself:
    - minval (in scalarEncoderArgs): The minimum possible value for this input
    - maxval (in scalarEncoderArgs): The maximum possible value for this input
    - name (in scalarEncoderArgs): The name of this input
    - clipInput (in scalarEncoderArgs): Clip input if the value exceed the min/max values. 
After this, these parameters will be writed into seperate files.

* If you don't want to change these parameters anymore, then you should just read them from files 
and comment out the codes below in this cell.
'''
encoder_params = {
   'scalarEncoder1Args':{
     "minval": 0,
     "maxval": 100,
     "w": 21,
     "periodic": False,
     "n": 50,
     "radius": 0,
     "resolution": 0,
     "name": "value",
     "verbosity": 0,
     "clipInput": False,
     "forced": False,
   },
       
   'dateEncoder1Args':{
     "season": 0,
     "dayOfWeek": 0,
     "weekend": 0,
     "holiday": 0,
     "timeOfDay": _TIMEOFDAY,
     "customDays": 0,
     "name": "time",
     "forced": False
   },
       
   'scalarEncoder2Args':{
     "minval": 0,
     "maxval": 100,
     "w": 21,
     "periodic": False,
     "n": 50,
     "radius": 0,
     "resolution": 0,
     "name": "value",
     "verbosity": 0,
     "clipInput": False,
     "forced": False,
   },
   
   'dateEncoder2Args':{
     "season": 0,
     "dayOfWeek": 0,
     "weekend": 0,
     "holiday": 0,
     "timeOfDay": _TIMEOFDAY,
     "customDays": 0,
     "name": "time",
     "forced": False
   }
}

# save encoder parameters
if not os.path.exists('./temp'):
    os.makedirs('./temp')
    
htm.setEncoderParams('./temp/encoders.json', encoder_params)

_SP_PARAMS = {
   'SP':{
       "spatialImp": "cpp",
       "globalInhibition": 1,
       "columnCount": 2048,
       "inputWidth": 0,
       "numActiveColumnsPerInhArea": 40,
       "seed": 1956,
       "potentialPct": 0.8,
       "synPermConnected": 0.1,
       "synPermActiveInc": 0.0001,
       "synPermInactiveDec": 0.0005,
       "boostStrength": 0.0,
   }
}

_TM_PARAMS = {
   'TM':{
       "columnCount": 2048,
       "cellsPerColumn": 32,
       "inputWidth": 2048,
       "seed": 1960,
       "temporalImp": "cpp",
       "newSynapseCount": 20,
       "maxSynapsesPerSegment": 32,
       "maxSegmentsPerCell": 128,
       "initialPerm": 0.21,
       "permanenceInc": 0.1,
       "permanenceDec": 0.1,
       "globalDecay": 0.0,
       "maxAge": 0,
       "minThreshold": 9,
       "activationThreshold": 12,
       "outputType": "normal",
       "pamLength": 3,
   }
}

# save SP and TM parameters
htm.setEncoderParams('./temp/SP.json', _SP_PARAMS)
htm.setEncoderParams('./temp/TM.json', _TM_PARAMS)

In [3]:
'''
Load parameters from json files and build data stream reader to read data from cache.
* Maybe we just hard-code these path? There are many of them for users to define.
'''

scalarEncoder1Args = htm.getEncoderParams('./temp/encoders.json', 'scalarEncoder1Args')
dateEncoder1Args = htm.getEncoderParams('./temp/encoders.json', 'dateEncoder1Args')
scalarEncoder2Args = htm.getEncoderParams('./temp/encoders.json', 'scalarEncoder2Args')
dateEncoder2Args = htm.getEncoderParams('./temp/encoders.json', 'dateEncoder2Args')
SPArgs = htm.getEncoderParams('./temp/SP.json', 'SP')
TMArgs = htm.getEncoderParams('./temp/TM.json', 'TM')

input01_recordParams = {
  "scalarEncoderArgs": scalarEncoder1Args,
  "dateEncoderArgs": dateEncoder1Args,
}

input02_recordParams = {
  "scalarEncoderArgs": scalarEncoder2Args,
  "dateEncoderArgs": dateEncoder2Args,
}

# generate the first batch data to cache
data_simulator.getBatchData2csv(_STREAM_BUFFER, './temp/datacache01.csv', './temp/datacache02.csv')

# define the data souce
streamReader1 = FileRecordStream(streamID = './temp/datacache01.csv')
streamReader2 = FileRecordStream(streamID = './temp/datacache02.csv')

In [4]:
'''
create networks and tell them the source of data for prediction
'''
network01 = htm.createNetwork(datasource=streamReader1, recordParams=input01_recordParams, spatialParams=SPArgs, temporalParams=TMArgs)
network02 = htm.createNetwork(datasource=streamReader2, recordParams=input02_recordParams, spatialParams=SPArgs, temporalParams=TMArgs)

In [5]:
'''
Looping network.run() to get iterative prediction from data cache.

* network.run(1) means run this network once on next row of data in csv cache.

Before you run this cell, I already called data_simulator.getBatchData2csv once in the 
above cell to initialize the first batch of data. After this, the new data will be required 
by the while loop below when it reach the end of cache.
'''
iteration = 0
    
def run_network_once():
        fed_in_data01, anomalyLikelihood1 = htm.run(network01)
        fed_in_data02, anomalyLikelihood2 = htm.run(network02)
        runTime = data_simulator.getRunningTime('./temp/datacache01.csv', (iteration % _STREAM_BUFFER) + 3)
        print 'Running time:', runTime, 'fed_in_data01:', fed_in_data01,' anomaly likelihood:', anomalyLikelihood1
        print 'Running time:', runTime, 'fed_in_data02', fed_in_data02,' anomaly likelihood:', anomalyLikelihood2, '\n'
        time.sleep(1)
        
while(iteration < 500):
    if iteration == 0:
        run_network_once()
    elif iteration % _STREAM_BUFFER != 0:
        run_network_once()
    else:
        network01.regions["sensor"].getSelf().dataSource.rewind()
        network02.regions["sensor"].getSelf().dataSource.rewind()
        data_simulator.getBatchData2csv(_STREAM_BUFFER, './temp/datacache01.csv', './temp/datacache02.csv')
        run_network_once()
    iteration += 1

'''
Save the model when breaking from while loop.

You should define your button in SENSEI to break this while loop and save the model 
because we usually set looping as while(1) instead of while(iteration < n)
'''
if not os.path.exists('./models'):
    os.makedirs('./models')
    
htm.save_network(network01, './models/network1.nta')
htm.save_network(network02, './models/network2.nta')

print '\nComplete.'

Running time: 2019-09-09 01:00:00 fed_in_data01: 8.4  anomaly likelihood: 0.5
Running time: 2019-09-09 01:00:00 fed_in_data02 45.5  anomaly likelihood: 0.5 

Running time: 2019-09-09 02:00:00 fed_in_data01: 9.2  anomaly likelihood: 0.5
Running time: 2019-09-09 02:00:00 fed_in_data02 45.5  anomaly likelihood: 0.5 

Running time: 2019-09-09 03:00:00 fed_in_data01: 11.9  anomaly likelihood: 0.5
Running time: 2019-09-09 03:00:00 fed_in_data02 45.5  anomaly likelihood: 0.5 

Running time: 2019-09-09 04:00:00 fed_in_data01: 11.5  anomaly likelihood: 0.5
Running time: 2019-09-09 04:00:00 fed_in_data02 45.5  anomaly likelihood: 0.5 

Running time: 2019-09-09 05:00:00 fed_in_data01: 9.2  anomaly likelihood: 0.5
Running time: 2019-09-09 05:00:00 fed_in_data02 45.5  anomaly likelihood: 0.5 

Running time: 2019-09-09 06:00:00 fed_in_data01: 7.8  anomaly likelihood: 0.5
Running time: 2019-09-09 06:00:00 fed_in_data02 45.5  anomaly likelihood: 0.5 

Running time: 2019-09-09 07:00:00 fed_in_data01: 1

Running time: 2019-09-11 05:00:00 fed_in_data01: 9.4  anomaly likelihood: 0.5
Running time: 2019-09-11 05:00:00 fed_in_data02 46.4  anomaly likelihood: 0.5 

Running time: 2019-09-11 06:00:00 fed_in_data01: 11.1  anomaly likelihood: 0.5
Running time: 2019-09-11 06:00:00 fed_in_data02 46.4  anomaly likelihood: 0.5 

Running time: 2019-09-11 07:00:00 fed_in_data01: 27.5  anomaly likelihood: 0.5
Running time: 2019-09-11 07:00:00 fed_in_data02 46.4  anomaly likelihood: 0.5 

Running time: 2019-09-11 08:00:00 fed_in_data01: 14.6  anomaly likelihood: 0.5
Running time: 2019-09-11 08:00:00 fed_in_data02 46.4  anomaly likelihood: 0.5 

Running time: 2019-09-11 09:00:00 fed_in_data01: 13.3  anomaly likelihood: 0.5
Running time: 2019-09-11 09:00:00 fed_in_data02 46.4  anomaly likelihood: 0.5 

Running time: 2019-09-11 10:00:00 fed_in_data01: 17.4  anomaly likelihood: 0.5
Running time: 2019-09-11 10:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-11 11:00:00 fed_in_data01

Running time: 2019-09-13 09:00:00 fed_in_data01: 3.1  anomaly likelihood: 0.5
Running time: 2019-09-13 09:00:00 fed_in_data02 46.6  anomaly likelihood: 0.5 

Running time: 2019-09-13 10:00:00 fed_in_data01: 4.9  anomaly likelihood: 0.5
Running time: 2019-09-13 10:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-13 11:00:00 fed_in_data01: 2.7  anomaly likelihood: 0.5
Running time: 2019-09-13 11:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-13 12:00:00 fed_in_data01: 7.8  anomaly likelihood: 0.5
Running time: 2019-09-13 12:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-13 13:00:00 fed_in_data01: 6.4  anomaly likelihood: 0.5
Running time: 2019-09-13 13:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-13 14:00:00 fed_in_data01: 4.9  anomaly likelihood: 0.5
Running time: 2019-09-13 14:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-13 15:00:00 fed_in_data01: 4.9

Running time: 2019-09-15 13:00:00 fed_in_data01: 4.1  anomaly likelihood: 0.5
Running time: 2019-09-15 13:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-15 14:00:00 fed_in_data01: 3.5  anomaly likelihood: 0.5
Running time: 2019-09-15 14:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-15 15:00:00 fed_in_data01: 10.5  anomaly likelihood: 0.5
Running time: 2019-09-15 15:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-15 16:00:00 fed_in_data01: 10.3  anomaly likelihood: 0.5
Running time: 2019-09-15 16:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-15 17:00:00 fed_in_data01: 4.9  anomaly likelihood: 0.5
Running time: 2019-09-15 17:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-15 18:00:00 fed_in_data01: 5.1  anomaly likelihood: 0.5
Running time: 2019-09-15 18:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-15 19:00:00 fed_in_data01: 9

Running time: 2019-09-17 17:00:00 fed_in_data01: 4.5  anomaly likelihood: 0.5
Running time: 2019-09-17 17:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-17 18:00:00 fed_in_data01: 5.7  anomaly likelihood: 0.5
Running time: 2019-09-17 18:00:00 fed_in_data02 46.4  anomaly likelihood: 0.5 

Running time: 2019-09-17 19:00:00 fed_in_data01: 9.2  anomaly likelihood: 0.5
Running time: 2019-09-17 19:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-17 20:00:00 fed_in_data01: 5.1  anomaly likelihood: 0.5
Running time: 2019-09-17 20:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-17 21:00:00 fed_in_data01: 4.5  anomaly likelihood: 0.5
Running time: 2019-09-17 21:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-17 22:00:00 fed_in_data01: 4.5  anomaly likelihood: 0.5
Running time: 2019-09-17 22:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-17 23:00:00 fed_in_data01: 4.7

Running time: 2019-09-19 21:00:00 fed_in_data01: 5.3  anomaly likelihood: 0.5
Running time: 2019-09-19 21:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-19 22:00:00 fed_in_data01: 3.7  anomaly likelihood: 0.5
Running time: 2019-09-19 22:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-19 23:00:00 fed_in_data01: 5.1  anomaly likelihood: 0.5
Running time: 2019-09-19 23:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-20 00:00:00 fed_in_data01: 5.7  anomaly likelihood: 0.5
Running time: 2019-09-20 00:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-20 01:00:00 fed_in_data01: 8.2  anomaly likelihood: 0.5
Running time: 2019-09-20 01:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-20 02:00:00 fed_in_data01: 5.7  anomaly likelihood: 0.5
Running time: 2019-09-20 02:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-20 03:00:00 fed_in_data01: 4.9

Running time: 2019-09-22 01:00:00 fed_in_data01: 7.4  anomaly likelihood: 0.5
Running time: 2019-09-22 01:00:00 fed_in_data02 46.4  anomaly likelihood: 0.5 

Running time: 2019-09-22 02:00:00 fed_in_data01: 4.9  anomaly likelihood: 0.5
Running time: 2019-09-22 02:00:00 fed_in_data02 46.4  anomaly likelihood: 0.5 

Running time: 2019-09-22 03:00:00 fed_in_data01: 6.4  anomaly likelihood: 0.5
Running time: 2019-09-22 03:00:00 fed_in_data02 46.4  anomaly likelihood: 0.5 

Running time: 2019-09-22 04:00:00 fed_in_data01: 11.7  anomaly likelihood: 0.5
Running time: 2019-09-22 04:00:00 fed_in_data02 46.4  anomaly likelihood: 0.5 

Running time: 2019-09-22 05:00:00 fed_in_data01: 10.2  anomaly likelihood: 0.5
Running time: 2019-09-22 05:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-22 06:00:00 fed_in_data01: 4.1  anomaly likelihood: 0.5
Running time: 2019-09-22 06:00:00 fed_in_data02 46.5  anomaly likelihood: 0.5 

Running time: 2019-09-22 07:00:00 fed_in_data01: 5

Running time: 2019-09-24 05:00:00 fed_in_data01: 3.1  anomaly likelihood: 0.5
Running time: 2019-09-24 05:00:00 fed_in_data02 46.7  anomaly likelihood: 0.5 

Running time: 2019-09-24 06:00:00 fed_in_data01: 3.9  anomaly likelihood: 0.5
Running time: 2019-09-24 06:00:00 fed_in_data02 46.7  anomaly likelihood: 0.5 

Running time: 2019-09-24 07:00:00 fed_in_data01: 6.8  anomaly likelihood: 0.5
Running time: 2019-09-24 07:00:00 fed_in_data02 46.7  anomaly likelihood: 0.5 

Running time: 2019-09-24 08:00:00 fed_in_data01: 4.9  anomaly likelihood: 0.5
Running time: 2019-09-24 08:00:00 fed_in_data02 46.7  anomaly likelihood: 0.5 

Running time: 2019-09-24 09:00:00 fed_in_data01: 14.2  anomaly likelihood: 0.5
Running time: 2019-09-24 09:00:00 fed_in_data02 46.7  anomaly likelihood: 0.5 

Running time: 2019-09-24 10:00:00 fed_in_data01: 7.0  anomaly likelihood: 0.5
Running time: 2019-09-24 10:00:00 fed_in_data02 46.7  anomaly likelihood: 0.5 

Running time: 2019-09-24 11:00:00 fed_in_data01: 5.

Running time: 2019-09-26 08:00:00 fed_in_data01: 4.5  anomaly likelihood: 0.724853
Running time: 2019-09-26 08:00:00 fed_in_data02 46.9  anomaly likelihood: 0.581105 

Running time: 2019-09-26 09:00:00 fed_in_data01: 5.2  anomaly likelihood: 0.666629
Running time: 2019-09-26 09:00:00 fed_in_data02 46.9  anomaly likelihood: 0.581105 

Running time: 2019-09-26 10:00:00 fed_in_data01: 4.1  anomaly likelihood: 0.666629
Running time: 2019-09-26 10:00:00 fed_in_data02 46.8  anomaly likelihood: 0.505918 

Running time: 2019-09-26 11:00:00 fed_in_data01: 5.1  anomaly likelihood: 0.625325
Running time: 2019-09-26 11:00:00 fed_in_data02 46.8  anomaly likelihood: 0.617784 

Running time: 2019-09-26 12:00:00 fed_in_data01: 3.9  anomaly likelihood: 0.625325
Running time: 2019-09-26 12:00:00 fed_in_data02 46.8  anomaly likelihood: 0.736296 

Running time: 2019-09-26 13:00:00 fed_in_data01: 7.0  anomaly likelihood: 0.706006
Running time: 2019-09-26 13:00:00 fed_in_data02 46.9  anomaly likelihood: 0.7

Running time: 2019-09-28 09:00:00 fed_in_data01: 7.4  anomaly likelihood: 0.886578
Running time: 2019-09-28 09:00:00 fed_in_data02 46.8  anomaly likelihood: 0.505918 

Running time: 2019-09-28 10:00:00 fed_in_data01: 10.9  anomaly likelihood: 0.875537
Running time: 2019-09-28 10:00:00 fed_in_data02 46.8  anomaly likelihood: 0.588047 

Running time: 2019-09-28 11:00:00 fed_in_data01: 8.8  anomaly likelihood: 0.838
Running time: 2019-09-28 11:00:00 fed_in_data02 46.8  anomaly likelihood: 0.624555 

Running time: 2019-09-28 12:00:00 fed_in_data01: 11.7  anomaly likelihood: 0.851256
Running time: 2019-09-28 12:00:00 fed_in_data02 46.8  anomaly likelihood: 0.505918 

Running time: 2019-09-28 13:00:00 fed_in_data01: 6.1  anomaly likelihood: 0.838
Running time: 2019-09-28 13:00:00 fed_in_data02 46.7  anomaly likelihood: 0.562476 

Running time: 2019-09-28 14:00:00 fed_in_data01: 6.3  anomaly likelihood: 0.809251
Running time: 2019-09-28 14:00:00 fed_in_data02 46.7  anomaly likelihood: 0.56247

### Example2. Use saved model to keep running on cache data

In [6]:
'''
Loading the saved model above and restore it to keep prediction & learning.

The data source will be the same as we defined above.

Notice that in HTM, the data source is associated with the network, and 
we should define that when we created the network, which is different from
LSTM where we feed data in actively. HTM network will use data generator 
(e.g., FileRecordStream) to automatically fetch the data from csv.

This is why we don't need to re-define our data source, spatial, and temporal params again
because these are the parts of 'model' we have saved.
'''

_USE_SAVED_MODEL = True
htm = HTM(use_saved_model = _USE_SAVED_MODEL)

scalarEncoder1Args = htm.getEncoderParams('./temp/encoders.json', 'scalarEncoder1Args')
dateEncoder1Args = htm.getEncoderParams('./temp/encoders.json', 'dateEncoder1Args')
scalarEncoder2Args = htm.getEncoderParams('./temp/encoders.json', 'scalarEncoder2Args')
dateEncoder2Args = htm.getEncoderParams('./temp/encoders.json', 'dateEncoder2Args')
SPArgs = htm.getEncoderParams('./temp/SP.json', 'SP')
TMArgs = htm.getEncoderParams('./temp/TM.json', 'TM')

input01_recordParams = {
  "scalarEncoderArgs": scalarEncoder1Args,
  "dateEncoderArgs": dateEncoder1Args,
}

input02_recordParams = {
  "scalarEncoderArgs": scalarEncoder2Args,
  "dateEncoderArgs": dateEncoder2Args,
}


model1 = './models/network1.nta'
model2 = './models/network2.nta'
network01 = htm.createNetwork(datasource=None, recordParams=input01_recordParams, spatialParams=None, temporalParams=None, model_path = model1)
network02 = htm.createNetwork(datasource=None, recordParams=input02_recordParams, spatialParams=None, temporalParams=None, model_path = model2)

# run the restored network on data from cache file
iteration = 0
    
def run_network_once():
        fed_in_data01, anomalyLikelihood1 = htm.run(network01)
        fed_in_data02, anomalyLikelihood2 = htm.run(network02)
        runTime = data_simulator.getRunningTime('./temp/datacache01.csv', (iteration % _STREAM_BUFFER) + 3)
        print 'Running time:', runTime, 'fed_in_data01:', fed_in_data01,' anomaly likelihood:', anomalyLikelihood1
        print 'Running time:', runTime, 'fed_in_data02', fed_in_data02,' anomaly likelihood:', anomalyLikelihood2, '\n'
        time.sleep(1)
        
while(iteration < 30):
    if iteration == 0:
        print 'frist part...'
        run_network_once()
    elif iteration % _STREAM_BUFFER != 0:
        print 'second part...'
        run_network_once()
    else:
        print 'update csv...'
        network01.regions["sensor"].getSelf().dataSource.rewind()
        network02.regions["sensor"].getSelf().dataSource.rewind()
        data_simulator.getBatchData2csv(_STREAM_BUFFER, './temp/datacache01.csv', './temp/datacache02.csv')
        run_network_once()
    iteration += 1
    
print '\nComplete.'

frist part...
Running time: 2019-09-29 11:00:00 fed_in_data01: 8.6  anomaly likelihood: 0.82502
Running time: 2019-09-29 11:00:00 fed_in_data02 47.0  anomaly likelihood: 0.877275 

second part...
Running time: 2019-09-29 12:00:00 fed_in_data01: 4.5  anomaly likelihood: 0.850322
Running time: 2019-09-29 12:00:00 fed_in_data02 47.0  anomaly likelihood: 0.889361 

second part...
Running time: 2019-09-29 13:00:00 fed_in_data01: 8.2  anomaly likelihood: 0.850322
Running time: 2019-09-29 13:00:00 fed_in_data02 47.0  anomaly likelihood: 0.889361 

second part...
Running time: 2019-09-29 14:00:00 fed_in_data01: 4.9  anomaly likelihood: 0.873058
Running time: 2019-09-29 14:00:00 fed_in_data02 47.0  anomaly likelihood: 0.889361 

second part...
Running time: 2019-09-29 15:00:00 fed_in_data01: 3.9  anomaly likelihood: 0.862009
Running time: 2019-09-29 15:00:00 fed_in_data02 47.0  anomaly likelihood: 0.889361 

second part...
Running time: 2019-09-29 16:00:00 fed_in_data01: 5.5  anomaly likelihood